Check that we have the appropriate reader for Sentinel-2:

In [1]:
from satpy import Scene
from satpy.writers import get_enhanced_image
import matplotlib.pyplot as plt

from satpy import available_readers
print(sorted(available_readers()))

/var/folders/x5/84lgyr6j2t78v5b4r9ztb78w0000gn/T/ipykernel_87137/2264912158.py:2: UserWarning: 'satpy.writers.get_enhanced_image' has been moved to 'satpy.enhancements.enhancer.get_enhanced_image'. Import from the new location instead (ex. 'from satpy.enhancements.enhancer import get_enhanced_image'). The old import paths will be removed in Satpy 1.0
  from satpy.writers import get_enhanced_image


['abi_l1b', 'abi_l1b_scmi', 'abi_l2_nc', 'acspo', 'agri_fy4a_l1', 'agri_fy4b_l1', 'ahi_hrit', 'ahi_hsd', 'ahi_l1b_gridded_bin', 'ahi_l2_nc', 'amsr2_l1b', 'amsr2_l2', 'amsr2_l2_gaasp', 'amsub_l1c_aapp', 'atms_l1b_nc', 'atms_sdr_hdf5', 'avhrr_l1b_aapp', 'avhrr_l1b_eps', 'avhrr_l1b_hrpt', 'avhrr_l1c_eum_gac_fdr_nc', 'aws1_mwr_l1b_nc', 'aws1_mwr_l1c_nc', 'camel_l3_nc', 'cmsaf-claas2_l2_nc', 'electrol_hrit', 'epic_l1b_h5', 'eps_sterna_mwr_l1b_nc', 'fci_l1c_nc', 'fci_l2_nc', 'generic_image', 'geocat', 'gerb_l2_hr_h5', 'ghi_l1', 'ghrsst_l2', 'glm_l2', 'goci2_l2_nc', 'goes-imager_hrit', 'goes-imager_nc', 'gpm_imerg', 'hsaf_h5', 'hy2_scat_l2b_h5', 'iasi_l2', 'iasi_l2_cdr_nc', 'iasi_ng_l2_nc', 'ici_l1b_nc', 'insat3d_img_l1b_h5', 'jami_hrit', 'li_l2_nc', 'maia', 'meris_nc_sen3', 'mhs_l1c_aapp', 'mimicTPW2_comp', 'mirs', 'msi_l1c_earthcare', 'msi_safe', 'msi_safe_l2a', 'msu_gsa_l1b', 'mtsat2-imager_hrit', 'multiple_sensors_isccpng_l1g_nc', 'mviri_l1b_fiduceo_nc', 'mwi_l1b_nc', 'mws_l1b_nc', 'nucap

In [2]:
from satpy import Scene
import os

safe_path = "S2B_MSIL2A_20220717T105629_N0510_R094_T30TXQ_20240717T092238.SAFE"

# Collect all files under the .SAFE directory
input_files = []
for root, dirs, files in os.walk(safe_path):
    for fname in files:
        if fname.endswith(".jp2") or fname.endswith(".xml"):
            input_files.append(os.path.join(root, fname))

# Create scene
scn = Scene(filenames=input_files, reader="msi_safe_l2a")

# Print available composites and datasets
print("Available composites:\n", scn.available_composite_names())
print("Available datasets:\n", scn.available_dataset_names())

Available composites:
 []
Available datasets:
 ['AOT', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'SCL', 'WVP', 'satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle']


In [15]:
scn.load(['B04', 'B03', 'B02'])
scn['B04'].data.shape

(10980, 10980)

In [20]:
print(scn.available_composite_names())

[]


In [37]:
from satpy.composites.core import GenericCompositor

class MyTrueColor(GenericCompositor):
    def __init__(self):
        super().__init__(
            name='true_color',
            common_channel_mask=True
        )

    def requires(self):
        # Return the band names your composite depends on
        return ['B04', 'B03', 'B02']

    def composite(self, datasets):
        """
        This method must return a new DataArray representing the composite.

        `datasets` is a dict mapping band names to xarray.DataArrays.
        """
        # Extract individual bands (red, green, blue)
        red = datasets['B04']
        green = datasets['B03']
        blue = datasets['B02']

        # Compose into an RGB stack
        import xarray as xr
        import numpy as np

        # Stack bands along a new 'band' dimension
        rgb = xr.concat([red, green, blue], dim='band')
        rgb = rgb.assign_coords(band=['red', 'green', 'blue'])

        # You can add additional processing here, e.g. normalization, gamma correction

        return rgb

# Usage:

# 1. Load required bands
scn.load(['B04', 'B03', 'B02'])

# 2. Instantiate the composite
true_color = MyTrueColor()

# 3. Apply composite to scene
rgb_data = true_color(scn)  # returns xarray.DataArray of composite

# 4. Use or visualize rgb_data as needed


TypeError: object of type 'Scene' has no len()

In [40]:
from satpy import Scene
from satpy.composites.core import GenericCompositor

# Step 1: build scene with band files
# assuming input_files is your list of .jp2/.xml files
scn = Scene(reader='msi_safe_l2a', filenames=input_files)

# Step 2: load the needed bands
scn.load(['B04', 'B03', 'B02'])
# If needed, also resample so all bands match resolution / grid (especially if mixing 10m / 20m)
# e.g.: scn.resample(scn.finest_area)

# Step 3: get the DataArrays for each band from the scene
da_red   = scn['B04']
da_green = scn['B03']
da_blue  = scn['B02']

# Step 4: instantiate the compositor
compositor = GenericCompositor('true_color', prerequisites=['B04', 'B03', 'B02'])

# Step 5: call the compositor with a list of DataArrays
composite_da = compositor([da_red, da_green, da_blue])

In [39]:
composite_da

<xarray.DataArray 'where-914e13e16e82ec017fcaf23dca3a8740' (bands: 3, y: 10980,
                                                            x: 10980)> Size: 1GB
dask.array<where, shape=(3, 10980, 10980), dtype=float32, chunksize=(1, 4096, 4096), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 88kB 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05
  * y            (y) float64 88kB 5e+06 5e+06 5e+06 ... 4.89e+06 4.89e+06
    spatial_ref  int64 8B 0
    crs          object 8B PROJCRS["WGS 84 / UTM zone 30N",BASEGEOGCRS["WGS 8...
  * bands        (bands) <U1 12B 'R' 'G' 'B'
Attributes: (12/15)
    resolution:              10
    sensor:                  msi
    standard_name:           boa_bidirectional_reflectance
    reader:                  msi_safe_l2a
    start_time:              2022-07-17 11:08:52.413098
    platform_name:           Sentinel-2B
    ...                      ...
    end_time:                2022-07-17 11:08:52.413098
    wavelength:              None
    name:                    true_color
    prerequisites:           ['B04', 'B03', 'B02']
    optional_prerequisites:  []
    mode:                    RGB

In [41]:
rgb_arr = composite_da.values

(3, 10980, 10980)